In [3]:
# download MNIST and read the data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
print("Train:", mnist.train.images.shape, "\nTest:",mnist.train.labels.shape)

Train: (55000, 784) 
Test: (55000, 10)


In [17]:
# define a model
import tensorflow as tf

# computation graph
x = tf.placeholder(tf.float32, [None, 784]) # None means a dimension can be of any length
y_tar = tf.placeholder(tf.float32, [None ,10])

W = tf.Variable(tf.truncated_normal([784,10]))
b = tf.Variable(tf.zeros([10]))

# loss function
# A straight way of doing it (not numeric stable):
# y = tf.nn.softmax(logits)
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_tar*tf.log(y), reduction_indices=[1]))
logits = tf.matmul(x,W) + b
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels = y_tar, logits = logits)
cross_entropy = tf.reduce_mean(cross_entropy)

# optimizer
learning_rate = 0.5
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [18]:
# training, the smaller the error margin, the better the model is
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# run the training step 1000 times!
for _ in range(1000):
    # stochastic training
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x:batch_xs, y_tar:batch_ys})

In [19]:
# evaluation
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y_tar,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# x, y are tf.placeholders
print(sess.run(accuracy, feed_dict={x:mnist.test.images, y_tar:mnist.test.labels}))

0.8883


In [33]:
# CNN
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))
def bias_variable(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))

# tf.nn.conv2d(input, filter/weight/kernel,strides,padding)
# input: [batch, in_height, in_width, in_channels]
# filter: [height, width, in_channels, out_channels]
# default stride: [batch,height,width,channels]
# padding: 'SAME' / 'VALID'
W_conv2d_1 = weight_variable([5,5,1,32]) # filter1
b_conv2d_1 = bias_variable([32])

# x_image = tf.placeholder(tf.float32,shape=[None,28,28,1])
x_image = tf.reshape(x,[-1,28,28,1])
z_conv2d_1 = tf.nn.conv2d(x_image,W_conv2d_1, strides=[1,1,1,1],padding='SAME')
h_conv2d_1 = tf.nn.relu(z_conv2d_1)
h_maxpool_1 = tf.nn.max_pool(h_conv2d_1, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')

W_conv2d_2 = weight_variable([5,5,32,64])
b_conv2d_2 = bias_variable([64])

z_conv2d_2 = tf.nn.conv2d(h_maxpool_1,W_conv2d_2,strides=[1,1,1,1],padding='SAME')
h_conv2d_2 = tf.nn.relu(z_conv2d_2)
h_maxpool_2 = tf.nn.max_pool(h_conv2d_2, ksize=[1,2,2,1], strides=[1,2,2,1],padding = 'SAME')

W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_maxpool_2_flat = tf.reshape(h_maxpool_2,[-1,7*7*64])
z_fc1 = tf.matmul(h_maxpool_2_flat, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(z_fc1)

# dropout layer
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# output layer: softmax
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

y_out = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_tar, logits=y_out)
cross_entropy = tf.reduce_mean(cross_entropy)

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_out,1), tf.argmax(y_tar,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0],y_tar:batch[1],keep_prob:1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x:batch[0],y_tar:batch[1], keep_prob:0.5})
    
    print('test accuracy %g' % accuracy.eval(feed_dict = {
        x:mnist.test.images, y_tar:mnist.test.labels, keep_prob:1.0
    }))

step 0, training accuracy 0.1
step 100, training accuracy 0.78
step 200, training accuracy 0.94
step 300, training accuracy 0.98
step 400, training accuracy 0.98
step 500, training accuracy 0.9
step 600, training accuracy 0.96
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 0.94
step 1000, training accuracy 0.98
step 1100, training accuracy 0.98
step 1200, training accuracy 1
step 1300, training accuracy 0.98
step 1400, training accuracy 0.98
step 1500, training accuracy 0.98
step 1600, training accuracy 1
step 1700, training accuracy 0.96
step 1800, training accuracy 1
step 1900, training accuracy 0.96
test accuracy 0.982
